In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
# parameter settings
is_plot = True
is_export = False

start=0
stride = 12
end = 120

landmarks_path = '../data/landmarks/refine_6kmh_braless_18markers_12fps.pkl'
meshes_path = '../data/meshes/6kmh_braless_26markers/'
test_landmarks_path = '../data/test/braless_random_landmarks.pkl'

# Data Loading

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object(landmarks_path)
landmarks.interp_field()

In [ ]:
from mesh4d import obj3d

o3_ls = obj3d.load_obj_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
    obj_type=obj3d.Obj3d_Deform
)

# Ultra-dense Motion Capture

In [ ]:
import time
from mesh4d import obj4d

start_time = time.time()

o4 = obj4d.Obj4d_Deform(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=True,
)
o4.add_obj(*o3_ls, k_nbr=100)
o4.load_markerset('landmarks', landmarks)

duration = time.time() - start_time

In [ ]:
print("computation time: {}".format(duration))

if is_export:
    utils.save_pkl_object(duration, 'output/nearest/duration.pkl')

In [ ]:
kps_source = landmarks.get_time_coord(0)
o4.vkps_track(kps_source, frame_id=0, k_nbr=10)

In [ ]:
from mesh4d import kps

vkps = o4.assemble_markerset(name='vkps')
diff = kps.MarkerSet.diff(vkps, landmarks)

In [ ]:
if is_export:
    utils.save_pkl_object(diff, 'output/nearest/diff.pkl')

In [ ]:
if is_plot:
    o4.show_deform_gif(output_folder='output/nearest/', filename='vkps', kps_names=('vkps', 'landmarks'))

# Randomly Selected Landmarks

In [ ]:
from mesh4d import utils

test_landmarks = utils.load_pkl_object(test_landmarks_path)
test_kps = test_landmarks.get_frame_coord(0)

In [ ]:
o4.vkps_track(test_kps, frame_id=0, name='vkps_random')
vkps_random = o4.assemble_markerset(name='vkps_random')
vkps_random.get_trace_length()

In [ ]:
if is_plot:
    o4.show_deform_gif(output_folder='output/nearest/', filename='vkps_random', kps_names=('vkps_random',))

# Full-field Deformation Estimation

Evenly sample points from the mesh and track its movement

In [ ]:
full_kps = o3_ls[0].get_sample_kps(10000)
o4.vkps_track(full_kps, frame_id=0, name='vkps_full')

In [ ]:
vkps_full = o4.assemble_markerset(name='vkps_full')
vkps_full.interp_field()

In [ ]:
_, starts, traces = vkps_full.get_trace_length()

if is_plot:
    o3_ls[0].show_with_value_mask(starts, traces, min_threshold=500, max_threshold=900)